In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
annotation_df = pd.read_csv('coco_annotations_socnosoc.csv')

In [4]:
nsd_stim_df = pd.read_csv('nsd_stim_info_merged.csv')

In [5]:
#Will need to add in all participant files here:
behav_files = ['../../../shared/NSD/nsddata/ppdata/subj01/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj02/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj03/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj04/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj05/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj06/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj07/behav/responses.tsv',
              '../../../shared/NSD/nsddata/ppdata/subj08/behav/responses.tsv',
              ]
first_file = True
for sub_tsv in behav_files:
    # curr_df = pd.read_csv(sub_tsv, sep='\t')
    # print(len(curr_df))
    # print(curr_df['SESSION'].nunique())
    if first_file != True:
        add_df = pd.read_csv(sub_tsv, sep='\t')
        behavioral_info_df = pd.concat([behavioral_info_df, add_df])
    else:
        behavioral_info_df = pd.read_csv(sub_tsv, sep='\t')
        first_file = False

In [6]:
len(behavioral_info_df)

213000

In [7]:
behavioral_info_df['73KID_0idx'] = behavioral_info_df['73KID'] - 1

In [8]:
nsd_ann_total = nsd_stim_df.merge(annotation_df, how='left', left_on='cocoId', right_on='id')
nsd_ann_total.drop('id', axis=1, inplace=True)

In [9]:
behavioral_info_df['trial_in_session'] = behavioral_info_df.groupby(['SESSION']).cumcount()

In [10]:
behav_ann_comb_df = behavioral_info_df.merge(nsd_ann_total, 
                    how='left', 
                    left_on='73KID_0idx',
                    right_on='nsdId')[['SUBJECT', 'SESSION', 'RUN', 'TRIAL', 
                                       'trial_in_session', 'cocoId', 'nsdId', 
                                       'flagged', 'BOLD5000', 'shared1000', 'file_name', 
                                       'coco_url','people_coco', 'people', 'faces',
                                       '73KID', '10KID', 'TIME', 'ISOLD', 'ISCORRECT', 
                                       'RT', 'CHANGEMIND', 'MEMORYRECENT', 'MEMORYFIRST',
                                       'ISOLDCURRENT', 'ISCORRECTCURRENT', 'TOTAL1', 
                                       'TOTAL2', 'BUTTON', 'MISSINGDATA', '73KID_0idx']]

In [11]:
behav_ann_comb_df.to_csv('annotations_behavioral_combined.csv')